Trying various models for sentiment classification

In [ ]:
%pip install transformers

In [ ]:
%pip install --upgrade protobuf tiktoken sentencepiece

In [ ]:
from transformers import pipeline

# RoBERTA, 7 labels, lightweight, fast-ish
emotion_model_1 = pipeline("text-classification", 
                           model="j-hartmann/emotion-english-distilroberta-base", 
                           return_all_scores=True) 
# T5, 6 labels, larger model, slower
emotion_model_2 = pipeline("text-classification", 
                           model="mrm8488/t5-base-finetuned-emotion", 
                           return_all_scores=True)
# DistilBERT, 6 labels, lightweight, fast, less accurate / more coarse
emotion_model_3 = pipeline("text-classification", 
                           model="bhadresh-savani/distilbert-base-uncased-emotion", 
                           return_all_scores=True)
# BERT, 6 labels, larger model, slower, need GPU
emotion_model_4 = pipeline("text-classification", 
                           model="nateraw/bert-base-uncased-emotion", 
                           return_all_scores=True)
# RoBERTA, 28 labels, larger model, slowest
emotion_model_5 = pipeline("text-classification",
                           model="SamLowe/roberta-base-go_emotions", 
                           return_all_scores=True)


In [6]:
def get_emotion_predictions(text):
    results = {
        "j-hartmann/emotion-english-distilroberta-base": emotion_model_1(text),
        "mrm8488/t5-base-finetuned-emotion": emotion_model_2(text),
        "bhadresh-savani/distilbert-base-uncased-emotion": emotion_model_3(text),
        "nateraw/bert-base-uncased-emotion": emotion_model_4(text),
        "SamLowe/roberta-base-go_emotions": emotion_model_5(text)
    }
    return results

texts = ["I failed my exam today.",                   # Sad 
         "I am so excited about my new job!",         # Joy, surpise
         "My friends are throwing me a party!",       # Joy, surprise
         "I threw up on my shoes.",                   # Disgust, sad
         "I'm worried my cat is sick.",               # Fear
         "My best friend is getting married.",        # Joy
         "The bed is warm.",                           # Neutral, joy
         "Hi",                                         # Neutral 
         "My boyfriend cheated on me.",                # Anger, sadness
         "My dog peed on the couch."                 # Disgust, anger
    ]

for text in texts:
    print(f"Input Text: {text}")
    predictions = get_emotion_predictions(text)
    for model_name, result in predictions.items():
        print(f"Model: {model_name}")
        for score in result[0]:
            print(f"  {score['label']}: {score['score']:.4f}")
        print()
    print("--------------------------------------------------\n")
    predictions = get_emotion_predictions(text)


Input Text: I failed my exam today.
Model: j-hartmann/emotion-english-distilroberta-base
  anger: 0.0196
  disgust: 0.0205
  fear: 0.0046
  joy: 0.0019
  neutral: 0.0032
  sadness: 0.9487
  surprise: 0.0015

Model: mrm8488/t5-base-finetuned-emotion
  LABEL_0: 0.7735
  LABEL_1: 0.2265

Model: bhadresh-savani/distilbert-base-uncased-emotion
  sadness: 0.1444
  joy: 0.0256
  love: 0.0013
  anger: 0.3166
  fear: 0.5086
  surprise: 0.0034

Model: nateraw/bert-base-uncased-emotion
  sadness: 0.6891
  joy: 0.0081
  love: 0.0021
  anger: 0.1591
  fear: 0.1373
  surprise: 0.0042

Model: SamLowe/roberta-base-go_emotions
  admiration: 0.0043
  amusement: 0.0017
  anger: 0.0043
  annoyance: 0.0486
  approval: 0.0109
  caring: 0.0024
  confusion: 0.0031
  curiosity: 0.0017
  desire: 0.0033
  disappointment: 0.7604
  disapproval: 0.0396
  disgust: 0.0046
  embarrassment: 0.0138
  excitement: 0.0026
  fear: 0.0030
  gratitude: 0.0014
  grief: 0.0041
  joy: 0.0053
  love: 0.0055
  nervousness: 0.0077


In [7]:
print(predictions)

{'j-hartmann/emotion-english-distilroberta-base': [[{'label': 'anger', 'score': 0.026681862771511078}, {'label': 'disgust', 'score': 0.9573454260826111}, {'label': 'fear', 'score': 0.003998803906142712}, {'label': 'joy', 'score': 0.0008786179823800921}, {'label': 'neutral', 'score': 0.0023822030052542686}, {'label': 'sadness', 'score': 0.007839730009436607}, {'label': 'surprise', 'score': 0.0008732916321605444}]], 'mrm8488/t5-base-finetuned-emotion': [[{'label': 'LABEL_0', 'score': 0.7700479030609131}, {'label': 'LABEL_1', 'score': 0.22995205223560333}]], 'bhadresh-savani/distilbert-base-uncased-emotion': [[{'label': 'sadness', 'score': 0.0032255405094474554}, {'label': 'joy', 'score': 0.002265207702293992}, {'label': 'love', 'score': 0.00047965513658709824}, {'label': 'anger', 'score': 0.15854603052139282}, {'label': 'fear', 'score': 0.8336644172668457}, {'label': 'surprise', 'score': 0.001819187542423606}]], 'nateraw/bert-base-uncased-emotion': [[{'label': 'sadness', 'score': 0.05529

In [ ]:
# Going with J-Hartmann's DistilRoBERTa model for sentiment classification

# Up-sampling the emotions + adding adjectives for diverse searches:
emotion_labels_upsampled = {
    "anger": ["angry", "furious", "mad", "annoyed", "irritated", "upset"],
    "disgust": ["gross", "ew", "grossed out", "puke"],
    "fear": ["scared", "worried", "terrified", "anxious"],
    "joy": ["happy", "excited", "joy", "dancing", "woohoo","yay", "yippee", "cute"],
    "neutral": ["neutral", "sleeping", "cool", "alright", "nice", "chill"],
    "sadness": ["sad", "unhappy", "crying", "sobbing", "lonely"],
    "surprise": ["surprised", "shocked", "startled"]
}